In [60]:
import sys
sys.path.append('../libs')  # Update this path according to the location of your 'dataset' module
import dataset
import preprocessing
import classes
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.preprocessing import RobustScaler
from sklearn import svm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [61]:
# Chosen Hyper Params

pca_components = 10
svm_cost = 1
svm_kernel = "poly"
scaler_q_min = 25
scaler_q_max = 75

In [62]:
X = dataset.data()
y = np.array(list(map(classes.label_to_class, dataset.labels_array())))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# X_test, X_validation, y_test, y_validation = train_test_split(X_test_validation, y_test_validation, test_size=0.25, random_state=999)

(samples, features, frames) = X_train.shape


In [63]:


# scaler = RobustScaler(quantile_range=(scaler_q_min, scaler_q_max))
# X_transposed = X_train.transpose(0,2,1)
# arr = np.reshape(X_transposed, (samples * frames, features))
# scaled = scaler.fit_transform(arr)

# pca = PCA(n_components=pca_components)
# reduced_X = pca.fit(scaled)

In [64]:
# flat_X = np.empty((samples, frames, pca_components))

# for i in range(samples):
#     flat_X[i] = pca.transform(scaler.transform(X_transposed[i]))

# flat_X = np.reshape(flat_X, (samples, frames * pca_components))
# print(flat_X.shape)

In [65]:
# clf = svm.SVC(cache_size=1000, verbose=True, class_weight='balanced', C = svm_cost, kernel=svm_kernel)
# clf.fit(flat_X, y_train)

In [66]:
# (samples, features, frames) = X_test.shape

# X_test_transposed = X_test.transpose(0,2,1)
# arr = np.reshape(X_test_transposed, (samples * frames, features))
# scaled = scaler.transform(arr)

# reduced_X_test = pca.transform(scaled)

# flat_X_test = np.empty((samples, frames, pca_components))

# for i in range(samples):
#     flat_X_test[i] = pca.transform(scaler.transform(X_test_transposed[i]))

# flat_X_test = np.reshape(flat_X_test, (samples, frames * pca_components))
# print(flat_X_test.shape)

# y_pred = clf.predict(flat_X_test)

In [67]:
# print(y_test[0:10])
# print(y_pred[0:10])
# print(np.array(list(classes.REVERSE_CLASSES.keys())))

# cm = confusion_matrix(y_test, y_pred, labels=np.array(list(classes.REVERSE_CLASSES.keys())))
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes.CLASSES)
# disp.plot()

In [72]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ShuffleSplit

class SVMClassifier(BaseEstimator):
     def __init__(self, pca_components = 10, svm_cost = 1, svm_kernel = "poly", scaler_q = 25):
          self.pca_components = pca_components 
          self.svm_cost = svm_cost 
          self.svm_kernel = svm_kernel
          self.scaler_q = scaler_q

          self.clf = None
          self.scaler = None
          self.pca = None

     def fit(self, X, y):

          print("Fitting now", X.shape)
          (samples, features, frames) = X.shape

          scaler_q_min = self.scaler_q 
          scaler_q_max = 100 - self.scaler_q

          self.scaler = RobustScaler(quantile_range=(scaler_q_min, scaler_q_max))
          X_transposed = X.transpose(0,2,1)
          arr = np.reshape(X_transposed, (samples * frames, features))
          scaled = self.scaler.fit_transform(arr)

          self.pca = PCA(n_components=self.pca_components)
          reduced_X = self.pca.fit(scaled)

          flat_X = np.empty((samples, frames, self.pca_components))

          for i in range(samples):
               flat_X[i] = self.pca.transform(self.scaler.transform(X_transposed[i]))

          flat_X = np.reshape(flat_X, (samples, frames * self.pca_components))

          self.clf = svm.SVC(cache_size=1000, verbose=True, class_weight='balanced', C = self.svm_cost, kernel=self.svm_kernel)
          self.clf.fit(flat_X, y)

          self.is_fitted_ = True
          return self

     def predict(self, X):

          if not self.is_fitted_:
               raise Exception("Not fitted")

          (samples, features, frames) = X.shape

          X_test_transposed = X.transpose(0,2,1)
          arr = np.reshape(X_test_transposed, (samples * frames, features))
          scaled = self.scaler.transform(arr)

          flat_X_test = np.empty((samples, frames, pca_components))

          for i in range(samples):
               flat_X_test[i] = self.pca.transform(self.scaler.transform(X_test_transposed[i]))

          flat_X_test = np.reshape(flat_X_test, (samples, frames * pca_components))

          y_pred = clf.predict(flat_X_test)

          return y_pred

In [69]:
# clf = SVMClassifier()
# clf.fit(X,y)
# y_pred = clf.predict(X_test)

In [73]:
parameters = {
    'svm_cost': [0.5, 1, 5], 
    "pca_components": [7, 10, 15],
    "svm_kernel": ["poly", "rbf"],
    "scaler_q": [25, 15, 5]
}

In [74]:

estimator = SVMClassifier()

clf = RandomizedSearchCV(estimator, parameters, n_iter=10, n_jobs=1,scoring="f1_weighted", verbose=True, cv=ShuffleSplit(n_splits=1, test_size=0.2))

clf.fit(X_train, y_train)


Fitting 1 folds for each of 10 candidates, totalling 10 fits
Fitting now (30800, 175, 44)
Fitting now (30800, 175, 44)
Fitting now (30800, 175, 44)
Fitting now (30800, 175, 44)
Fitting now (30800, 175, 44)
Fitting now (30800, 175, 44)
Fitting now (30800, 175, 44)
Fitting now (30800, 175, 44)
Fitting now (30800, 175, 44)
Fitting now (30800, 175, 44)


ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/simon/.conda/envs/mlpc-2024/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/tmp/ipykernel_2107/4154501050.py", line 37, in fit
    flat_X[i] = self.pca.transform(scaler.transform(X_transposed[i]))
                                   ^^^^^^
NameError: name 'scaler' is not defined. Did you mean: 'self.scaler'?


In [ ]:
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred, labels=np.array(list(classes.REVERSE_CLASSES.keys())))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes.CLASSES)
disp.plot()